# 1st Assignment

Marina Samprovalaki f3322310  </n>

Petros Chanas f3322314


In the 1st Assignment we had to create some n-gram language models in order to implement a context-aware spelling corrector

We assume that each sentence starts with the
pseudo-token *start* (or two pseudo-tokens *start1*, *start2* for the trigram model) and ends with the pseudo-token *end*.


# Libraries Installation

In [ ]:
!pip install -U nltk

In [ ]:
pip install tabulate

In [ ]:
import nltk

nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer

import random
import math

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

# Selecting a corpus

A corpus is a collection of text that serves as a valuable resource for language analysis, training models, and conducting research. As per the assignment's description, we recommend choosing a corpus from the [link](http://www.nltk.org/). We selected the Alice's Adventures in Wonderland, a novel by Lewis Carroll. We found this corpus in the Gutenberg Corpus​​ NLTK.

In [ ]:
nltk.download("gutenberg")
from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
# Access and print the text of "Alice's Adventures in Wonderland"
alice_text = gutenberg.raw("carroll-alice.txt")
print(alice_text[:500])  # Print the first 500 characters as a sample


[Alice's Adventures in Wonderland by Lewis Carroll 1865]

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Alice 'without pictures or
conversation?'

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy an


## Lowercases

In [ ]:
# Convert to lowercase and split by newlines
alice_text = alice_text.lower().split('\n')
alice_text

["[alice's adventures in wonderland by lewis carroll 1865]",
 '',
 'chapter i. down the rabbit-hole',
 '',
 'alice was beginning to get very tired of sitting by her sister on the',
 'bank, and of having nothing to do: once or twice she had peeped into the',
 'book her sister was reading, but it had no pictures or conversations in',
 "it, 'and what is the use of a book,' thought alice 'without pictures or",
 "conversation?'",
 '',
 'so she was considering in her own mind (as well as she could, for the',
 'hot day made her feel very sleepy and stupid), whether the pleasure',
 'of making a daisy-chain would be worth the trouble of getting up and',
 'picking the daisies, when suddenly a white rabbit with pink eyes ran',
 'close by her.',
 '',
 'there was nothing so very remarkable in that; nor did alice think it so',
 "very much out of the way to hear the rabbit say to itself, 'oh dear!",
 "oh dear! i shall be late!' (when she thought it over afterwards, it",
 'occurred to her that she oug

# Dividing the dataset

To facilitate the development and evaluation of n-gram models, it is essential to divide our corpus into three distinct subsets. The train set, used for training, the valid set for validation, and the test set for conducting tests, have been created with shuffling to prevent biases.


In [ ]:
import random

# Define the ratios for splitting
train_ratio = 0.7  # 70% for training
valid_ratio = 0.15  # 15% for validation
test_ratio = 0.15  # 15% for testing


# Shuffle the alice_text to ensure randomness
random.shuffle(alice_text)

# Calculate the split points based on the ratios
total_alice_text = len(alice_text)
train_split = int(total_alice_text * train_ratio)
valid_split = train_split + int(total_alice_text * valid_ratio)

# Split the alice_text into subsets
train_set = alice_text[:train_split]
valid_set = alice_text[train_split:valid_split]
test_set = alice_text[valid_split:]

# Join the alice_text back into text if needed
train_text = '\n'.join(train_set)
valid_text = '\n'.join(valid_set)
test_text = '\n'.join(test_set)


In [ ]:
train_text

'left alive!\'\ndispute with the queen, but she knew that it might happen any minute,\nleading right into it. \'that\'s very curious!\' she thought. \'but\n\n\'i don\'t like the look of it at all,\' said the king: \'however, it may\n\n\'then it wasn\'t very civil of you to offer it,\' said alice angrily.\n    pray how did you manage to do it?\'\nnot quite sure whether it was good manners for her to speak first, \'why\nsentence first!\'\n  \'\'tis the voice of the lobster; i heard him declare,\nbe growing small again.\' she got up and went to the table to measure\nso closely against her foot, that there was hardly room to open her\n\'that was a narrow escape!\' said alice, a good deal frightened at the\n\n\nhim,) \'i\'ll give him sixpence. _i_ don\'t believe there\'s an atom of\n\nmy fur and whiskers! she\'ll get me executed, as sure as ferrets are\n\nconversation. \'are you--are you fond--of--of dogs?\' the mouse did not\n\n   soup of the evening, beautiful soup!\nfor any lesson-books!

In [ ]:
valid_text

'all of them bowed low.\nhe was an old crab, he was.\'\n   \'beautiful soup! who cares for fish,\n\nthe royal children; there were ten of them, and the little dears came\ncautiously: \'but i don\'t understand. where did they draw the treacle\nnever before seen a rabbit with either a waistcoat-pocket, or a watch\n\n\n\nfor some minutes the whole court was in confusion, getting the dormouse\nwithout even waiting to put his shoes on.\n\'yes, but i grow at a reasonable pace,\' said the dormouse: \'not in that\nthe gryphon added \'come, let\'s hear some of your adventures.\'\nsame when i got up this morning? i almost think i can remember feeling a\ncrown. william\'s conduct at first was moderate. but the insolence of his\nabout the whiting!\'\nbegan ordering people about like that!\'\n\nsight, they were getting so far off). \'oh, my poor little feet, i wonder\non.\'\n\n\n\noh! won\'t she be savage if i\'ve kept her waiting!\' alice felt so\nthings are "much of a muchness"--did you ever see 

In [ ]:
test_text

'\'but i must be on the look-out for serpents night and day! why, i\nwho turned pale and fidgeted.\n\'yes, i think you\'d better leave off,\' said the gryphon: and alice was\nthe mock turtle sighed deeply, and began, in a voice sometimes choked\n\n\nit twelve? i--\'\nsavage when they met in the kitchen.\nbegged the mouse to tell them something more.\nremembering her own child-life, and the happy summer days.\n\'and what are they made of?\' alice asked in a tone of great curiosity.\nminutes it puffed away without speaking, but at last it unfolded its\n\n\nthen they both bowed low, and their curls got entangled together.\n\nthing before, and he wasn\'t going to begin at his time of life.\nturned the corner, but the rabbit was no longer to be seen: she found\n\nset to work at once to eat some of the other bit. her chin was pressed\nas all that.\'\nthey had not gone far before they saw the mock turtle in the distance,\n\n\n\n\'very,\' said alice: \'--where\'s the duchess?\'\nit sounded an 

# Train Subset Preprocessing

In [ ]:
train_text

'left alive!\'\ndispute with the queen, but she knew that it might happen any minute,\nleading right into it. \'that\'s very curious!\' she thought. \'but\n\n\'i don\'t like the look of it at all,\' said the king: \'however, it may\n\n\'then it wasn\'t very civil of you to offer it,\' said alice angrily.\n    pray how did you manage to do it?\'\nnot quite sure whether it was good manners for her to speak first, \'why\nsentence first!\'\n  \'\'tis the voice of the lobster; i heard him declare,\nbe growing small again.\' she got up and went to the table to measure\nso closely against her foot, that there was hardly room to open her\n\'that was a narrow escape!\' said alice, a good deal frightened at the\n\n\nhim,) \'i\'ll give him sixpence. _i_ don\'t believe there\'s an atom of\n\nmy fur and whiskers! she\'ll get me executed, as sure as ferrets are\n\nconversation. \'are you--are you fond--of--of dogs?\' the mouse did not\n\n   soup of the evening, beautiful soup!\nfor any lesson-books!

## Vocabulary

In [ ]:
import nltk
from collections import Counter

# Tokenize the text
words = nltk.word_tokenize(train_text)

# Count word frequencies
word_frequencies = Counter(words)

# Sort word frequencies by count in descending order
sorted_word_frequencies = dict(sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True))

# Print sorted word frequencies
for word, frequency in sorted_word_frequencies.items():
    print(f"{word}: {frequency}")


,: 1685
the: 1115
': 909
.: 654
and: 573
to: 485
a: 461
she: 378
it: 375
i: 364
of: 340
!: 321
said: 321
alice: 280
you: 250
was: 249
in: 243
that: 209
as: 191
--: 188
her: 169
:: 164
at: 151
;: 148
with: 138
's: 138
n't: 134
on: 134
had: 133
?: 132
all: 111
be: 97
very: 96
for: 95
little: 92
this: 91
not: 89
so: 89
do: 88
but: 86
they: 86
he: 85
is: 81
out: 81
down: 78
one: 71
what: 68
his: 67
up: 64
there: 64
about: 63
then: 62
could: 61
them: 60
would: 60
again: 59
were: 59
herself: 59
queen: 58
like: 58
went: 56
if: 56
or: 56
'': 55
know: 55
into: 54
have: 54
thought: 50
see: 50
off: 48
no: 48
when: 48
its: 48
'm: 47
your: 47
'it: 45
me: 44
time: 43
king: 42
did: 42
): 42
an: 41
think: 41
their: 40
*: 40
(: 39
by: 39
head: 39
voice: 38
'll: 38
my: 38
hatter: 38
'and: 38
much: 38
quite: 37
more: 37
go: 37
how: 36
first: 36
him: 36
some: 36
who: 36
turtle: 36
now: 36
gryphon: 35
here: 35
thing: 35
mock: 35
are: 34
way: 34
which: 34
mouse: 33
just: 33
'you: 33
only: 32
over: 32
great:

In [ ]:
# Filter words with frequency > 10
filtered_word_frequencies = {word: frequency for word, frequency in word_frequencies.items() if frequency > 10}

# Sort word frequencies by count in descending order
sorted_word_frequencies = dict(sorted(filtered_word_frequencies.items(), key=lambda item: item[1], reverse=True))


In [ ]:
vocab = filtered_word_frequencies
vocab

{'left': 12,
 '!': 321,
 "'": 909,
 'with': 138,
 'the': 1115,
 'queen': 58,
 ',': 1685,
 'but': 86,
 'she': 378,
 'that': 209,
 'it': 375,
 'might': 20,
 'any': 21,
 'minute': 17,
 'right': 23,
 'into': 54,
 '.': 654,
 "'that": 22,
 "'s": 138,
 'very': 96,
 'curious': 14,
 'thought': 50,
 "'but": 27,
 'i': 364,
 'do': 88,
 "n't": 134,
 'like': 58,
 'look': 22,
 'of': 340,
 'at': 151,
 'all': 111,
 'said': 321,
 'king': 42,
 ':': 164,
 'was': 249,
 'you': 250,
 'to': 485,
 'alice': 280,
 'how': 36,
 'did': 42,
 '?': 132,
 'not': 89,
 'quite': 37,
 'sure': 14,
 'good': 20,
 'for': 95,
 'her': 169,
 'first': 36,
 "'why": 13,
 "''": 55,
 'voice': 38,
 ';': 148,
 'heard': 24,
 'him': 36,
 'be': 97,
 'again': 59,
 'got': 29,
 'up': 64,
 'and': 573,
 'went': 56,
 'table': 13,
 'so': 89,
 'there': 64,
 'a': 461,
 ')': 42,
 "'ll": 38,
 'an': 41,
 'my': 38,
 'get': 29,
 'me': 44,
 'as': 191,
 'are': 34,
 '--': 188,
 'mouse': 33,
 'soup': 16,
 'should': 27,
 'them': 60,
 'last': 20,
 'they': 86,

In [ ]:
len(vocab)

271

## OOV

In [ ]:
filtered_words = [word if word_frequencies[word] > 10 else "UNK" for word in words]
filtered_words

['left',
 'UNK',
 '!',
 "'",
 'UNK',
 'with',
 'the',
 'queen',
 ',',
 'but',
 'she',
 'UNK',
 'that',
 'it',
 'might',
 'UNK',
 'any',
 'minute',
 ',',
 'UNK',
 'right',
 'into',
 'it',
 '.',
 "'that",
 "'s",
 'very',
 'curious',
 '!',
 "'",
 'she',
 'thought',
 '.',
 "'but",
 "'",
 'i',
 'do',
 "n't",
 'like',
 'the',
 'look',
 'of',
 'it',
 'at',
 'all',
 ',',
 "'",
 'said',
 'the',
 'king',
 ':',
 'UNK',
 ',',
 'it',
 'UNK',
 'UNK',
 'it',
 'was',
 "n't",
 'very',
 'UNK',
 'of',
 'you',
 'to',
 'UNK',
 'it',
 ',',
 "'",
 'said',
 'alice',
 'UNK',
 '.',
 'UNK',
 'how',
 'did',
 'you',
 'UNK',
 'to',
 'do',
 'it',
 '?',
 "'",
 'not',
 'quite',
 'sure',
 'UNK',
 'it',
 'was',
 'good',
 'UNK',
 'for',
 'her',
 'to',
 'UNK',
 'first',
 ',',
 "'why",
 'UNK',
 'first',
 '!',
 "'",
 "''",
 'UNK',
 'the',
 'voice',
 'of',
 'the',
 'UNK',
 ';',
 'i',
 'heard',
 'him',
 'UNK',
 ',',
 'be',
 'UNK',
 'UNK',
 'again',
 '.',
 "'",
 'she',
 'got',
 'up',
 'and',
 'went',
 'to',
 'the',
 'table',
 

In [ ]:
# Count OOV words
oov_count = filtered_words.count("UNK")

print("Number of OOV words:", oov_count)

Number of OOV words: 4923


In [ ]:
train_text = " ".join([word if word_frequencies[word] > 10 else "UNK" for word in words])
train_text

"left UNK ! ' UNK with the queen , but she UNK that it might UNK any minute , UNK right into it . 'that 's very curious ! ' she thought . 'but ' i do n't like the look of it at all , ' said the king : UNK , it UNK UNK it was n't very UNK of you to UNK it , ' said alice UNK . UNK how did you UNK to do it ? ' not quite sure UNK it was good UNK for her to UNK first , 'why UNK first ! ' '' UNK the voice of the UNK ; i heard him UNK , be UNK UNK again . ' she got up and went to the table to UNK so UNK UNK her UNK , that there was UNK UNK to UNK her 'that was a UNK UNK ! ' said alice , a good UNK UNK at the him , ) ' i 'll UNK him UNK . UNK do n't UNK there 's an UNK of my UNK and UNK ! she 'll get me UNK , as sure as UNK are UNK . UNK you -- are you UNK -- of -- of UNK ? ' the mouse did not soup of the UNK , UNK soup ! for any UNK ! ' UNK , should be UNK of them at last , they must UNK come UNK down from UNK that , ' UNK the hatter , ' i UNK some more UNK -- ' the UNK of that is -- '' UNK o

In [ ]:
def replace_oov_in_text(text):
    # Tokenize the input text into words
    tokens = word_tokenize(text)

    # Replace OOV words with "UNK" and join them back into plain text
    plain_text = ' '.join(["UNK" if token not in vocab else token for token in tokens])

    return plain_text

In [ ]:
valid_text = replace_oov_in_text(valid_text)
valid_text

"all of them UNK UNK . he was an old UNK , he was . ' UNK soup ! who UNK for UNK , the UNK UNK ; there were UNK of them , and the little UNK came UNK : 'but i do n't UNK . UNK did they UNK the UNK never before UNK a rabbit with UNK a UNK , or a UNK for some UNK the UNK court was in UNK , getting the dormouse without even UNK to put his UNK on . UNK , but i UNK at a UNK UNK , ' said the dormouse : UNK in that the gryphon added UNK , let 's UNK some of your UNK . ' same when i got up this UNK ? i UNK think i can UNK UNK a UNK . UNK 's UNK at first was UNK . but the UNK of his about the UNK ! ' began UNK UNK about like that ! ' UNK , they were getting so UNK off ) . 'oh , my poor little feet , i UNK on . ' oh ! wo n't she be UNK if i 've UNK her UNK ! ' alice felt so things are `` much of a UNK '' -- did you ever see such a thing as a UNK , i think ? ' tried to get her head down to them , and was UNK to find that her her UNK , and had come to the UNK UNK , that UNK you go and UNK a little

In [ ]:
test_text = replace_oov_in_text(test_text)
test_text

"'but i must be on the UNK for UNK UNK and day ! why , i who turned UNK and UNK . UNK , i think you 'd UNK UNK off , ' said the gryphon : and alice was the mock turtle UNK UNK , and began , in a voice UNK UNK it UNK ? i -- ' UNK when they UNK in the UNK . UNK the mouse to tell them something more . UNK her UNK UNK , and the UNK UNK UNK . 'and what are they made of ? ' alice asked in a tone of great UNK . UNK it UNK away without UNK , but at last it UNK its then they UNK UNK UNK , and their UNK got UNK UNK . thing before , and he was n't going to UNK at his time of UNK . turned the UNK , but the rabbit was no UNK to be UNK : she found UNK to UNK at once to eat some of the other bit . her UNK was UNK as all that . ' they had not UNK UNK before they saw the mock turtle in the UNK , UNK , ' said alice : ' -- UNK 's the duchess ? ' it UNK an UNK UNK , no UNK , and very UNK and UNK alice UNK her UNK , and began : -- UNK of UNK UNK that UNK UNK UNK her . 'and just as i 'd UNK the UNK UNK in t

## Tokenization in sentences

Dividing a block of text into individual sentences.

In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

sentences = sent_tokenize(train_text)

for sent in sentences:
    print(sent)
    print("_________________")

left UNK ! '
_________________
UNK with the queen , but she UNK that it might UNK any minute , UNK right into it .
_________________
'that 's very curious ! '
_________________
she thought .
_________________
'but ' i do n't like the look of it at all , ' said the king : UNK , it UNK UNK it was n't very UNK of you to UNK it , ' said alice UNK .
_________________
UNK how did you UNK to do it ? '
_________________
not quite sure UNK it was good UNK for her to UNK first , 'why UNK first ! '
_________________
'' UNK the voice of the UNK ; i heard him UNK , be UNK UNK again . '
_________________
she got up and went to the table to UNK so UNK UNK her UNK , that there was UNK UNK to UNK her 'that was a UNK UNK ! '
_________________
said alice , a good UNK UNK at the him , ) ' i 'll UNK him UNK .
_________________
UNK do n't UNK there 's an UNK of my UNK and UNK !
_________________
she 'll get me UNK , as sure as UNK are UNK .
_________________
UNK you -- are you UNK -- of -- of UNK ? '
______

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



he looked at the gryphon as if he thought it had some UNK of UNK and UNK UNK UNK .
_________________
the poor little UNK , bill , was UNK little UNK of her head to UNK back the UNK UNK that and UNK to UNK UNK one of the UNK UNK him .
_________________
a UNK ; but , on being UNK by the hatter , it UNK up again with but the UNK replied `` too UNK , too UNK ! ''
_________________
and gave a look UNK -- 'the UNK ! '
_________________
UNK the gryphon , with a UNK into the air .
_________________
UNK it what you like , ' said the cat .
_________________
'do you UNK UNK with the about half no time !
_________________
take your UNK ! '
_________________
she gave me a good UNK , UNK my UNK , ' said his UNK , ' i took to the UNK , ' i 've been to a UNK , too , ' said alice ; 'you UNK n't be so UNK UNK to UNK up one of the UNK of the table , but it was too UNK ; UNK on it were white , but there were three UNK at it , UNK 'you UNK to be UNK of UNK , ' said alice , ' a great UNK like the UNK when 

## Tokenization (word level)

NLTK offers various methods for splitting your text into individual words or tokens. In this case, we will utilize the TweetTokenizer for this purpose.

In [ ]:
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer

tweet_wt = TweetTokenizer()

sentences_tokenized = []

for sent in sentences:
    sent_tok = tweet_wt.tokenize(sent)
    sentences_tokenized.append(sent_tok)
    print(sent_tok)
    print("_________________")


['left', 'UNK', '!', "'"]
_________________
['UNK', 'with', 'the', 'queen', ',', 'but', 'she', 'UNK', 'that', 'it', 'might', 'UNK', 'any', 'minute', ',', 'UNK', 'right', 'into', 'it', '.']
_________________
["'", 'that', "'", 's', 'very', 'curious', '!', "'"]
_________________
['she', 'thought', '.']
_________________
["'", 'but', "'", 'i', 'do', "n't", 'like', 'the', 'look', 'of', 'it', 'at', 'all', ',', "'", 'said', 'the', 'king', ':', 'UNK', ',', 'it', 'UNK', 'UNK', 'it', 'was', "n't", 'very', 'UNK', 'of', 'you', 'to', 'UNK', 'it', ',', "'", 'said', 'alice', 'UNK', '.']
_________________
['UNK', 'how', 'did', 'you', 'UNK', 'to', 'do', 'it', '?', "'"]
_________________
['not', 'quite', 'sure', 'UNK', 'it', 'was', 'good', 'UNK', 'for', 'her', 'to', 'UNK', 'first', ',', "'", 'why', 'UNK', 'first', '!', "'"]
_________________
["'", "'", 'UNK', 'the', 'voice', 'of', 'the', 'UNK', ';', 'i', 'heard', 'him', 'UNK', ',', 'be', 'UNK', 'UNK', 'again', '.', "'"]
_________________
['she', 'got',

In [ ]:
from pprint import pprint

tokens = tweet_wt.tokenize(train_text)
tokens

['left',
 'UNK',
 '!',
 "'",
 'UNK',
 'with',
 'the',
 'queen',
 ',',
 'but',
 'she',
 'UNK',
 'that',
 'it',
 'might',
 'UNK',
 'any',
 'minute',
 ',',
 'UNK',
 'right',
 'into',
 'it',
 '.',
 "'",
 'that',
 "'",
 's',
 'very',
 'curious',
 '!',
 "'",
 'she',
 'thought',
 '.',
 "'",
 'but',
 "'",
 'i',
 'do',
 "n't",
 'like',
 'the',
 'look',
 'of',
 'it',
 'at',
 'all',
 ',',
 "'",
 'said',
 'the',
 'king',
 ':',
 'UNK',
 ',',
 'it',
 'UNK',
 'UNK',
 'it',
 'was',
 "n't",
 'very',
 'UNK',
 'of',
 'you',
 'to',
 'UNK',
 'it',
 ',',
 "'",
 'said',
 'alice',
 'UNK',
 '.',
 'UNK',
 'how',
 'did',
 'you',
 'UNK',
 'to',
 'do',
 'it',
 '?',
 "'",
 'not',
 'quite',
 'sure',
 'UNK',
 'it',
 'was',
 'good',
 'UNK',
 'for',
 'her',
 'to',
 'UNK',
 'first',
 ',',
 "'",
 'why',
 'UNK',
 'first',
 '!',
 "'",
 "'",
 "'",
 'UNK',
 'the',
 'voice',
 'of',
 'the',
 'UNK',
 ';',
 'i',
 'heard',
 'him',
 'UNK',
 ',',
 'be',
 'UNK',
 'UNK',
 'again',
 '.',
 "'",
 'she',
 'got',
 'up',
 'and',
 'went',
 

## Tokens Frequency

Token frequency simply counts how often individual words or phrases appear in a text or dataset.

In [ ]:
# Frequency distribution
N = 100
count = nltk.FreqDist(tokens)
print('{} most common tokens: \n')

pprint(count.most_common(N))

{} most common tokens: 

[('UNK', 4923),
 (',', 1685),
 ("'", 1607),
 ('the', 1130),
 ('.', 654),
 ('and', 611),
 ('to', 485),
 ('a', 461),
 ('it', 420),
 ('she', 378),
 ('-', 376),
 ('i', 364),
 ('of', 340),
 ('!', 321),
 ('said', 321),
 ('you', 283),
 ('alice', 280),
 ('was', 249),
 ('in', 243),
 ('that', 231),
 ('as', 191),
 ('her', 169),
 (':', 164),
 ('at', 151),
 (';', 148),
 ('with', 138),
 ('s', 138),
 ("n't", 134),
 ('on', 134),
 ('had', 133),
 ('?', 132),
 ('but', 113),
 ('all', 111),
 ('do', 100),
 ('be', 97),
 ('very', 96),
 ('for', 95),
 ('what', 94),
 ('little', 92),
 ('this', 91),
 ('not', 89),
 ('so', 89),
 ('they', 86),
 ('he', 85),
 ('is', 81),
 ('out', 81),
 ('down', 78),
 ('one', 71),
 ('if', 70),
 ('his', 67),
 ('up', 64),
 ('there', 64),
 ('about', 63),
 ('then', 62),
 ('could', 61),
 ('them', 60),
 ('would', 60),
 ('again', 59),
 ('were', 59),
 ('herself', 59),
 ('no', 59),
 ('queen', 58),
 ('like', 58),
 ('went', 56),
 ('or', 56),
 ('know', 55),
 ('into', 54),
 

# Create and count n-grams frequency with zip function

The [zip function](https://www.w3schools.com/python/ref_func_zip.asp) in Python is used to combine two or more iterables (e.g., lists, tuples) element-wise into tuples. When used to create n-grams, you can use it to pair words or tokens from a text to form n-grams by sliding a "window" of a specific size over the sequence

In [ ]:
unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for sent in sentences_tokenized:

    # Update the unigram counter
    unigram_counter.update([(gram,) for gram in sent])

    # Update the bigram counter
    bigram_pad_sent = ["<start>"] + sent +  ["<end>"]
    bigram_counter.update([(gram1, gram2) for gram1, gram2 in zip(bigram_pad_sent, bigram_pad_sent[1:])])

    # Update the trigram counter
    trigram_pad_sent = ["<start1>", "<start2>"] + sent + ["<end>"]
    trigram_counter.update([(gram1, gram2, gram3) for gram1, gram2, gram3 in zip(trigram_pad_sent, trigram_pad_sent[1:], trigram_pad_sent[2:])])


pprint(unigram_counter.most_common(10))
pprint(bigram_counter.most_common(10))
pprint(trigram_counter.most_common(10))

[(('UNK',), 4923),
 ((',',), 1685),
 (("'",), 1607),
 (('the',), 1130),
 (('.',), 654),
 (('and',), 611),
 (('to',), 485),
 (('a',), 461),
 (('it',), 420),
 (('she',), 378)]
[(('UNK', ','), 738),
 (('UNK', 'UNK'), 736),
 (("'", '<end>'), 490),
 (('the', 'UNK'), 488),
 (('.', '<end>'), 469),
 ((',', "'"), 367),
 ((',', 'and'), 305),
 (('UNK', '.'), 300),
 ((',', 'UNK'), 277),
 (('<start>', 'UNK'), 264)]
[(('<start1>', '<start2>', 'UNK'), 264),
 (('UNK', '.', '<end>'), 206),
 (('!', "'", '<end>'), 205),
 (('.', "'", '<end>'), 163),
 (('UNK', ',', "'"), 163),
 (('UNK', ',', 'and'), 157),
 ((',', "'", 'said'), 151),
 (('UNK', 'UNK', ','), 135),
 (('UNK', ',', 'UNK'), 125),
 (('UNK', '!', "'"), 111)]


#  Calculate bi-gram probability

### $ P(w_2|w_1) = \frac{C(w_1,w_2) + \alpha}{C(w_1) + \alpha \cdot|V|} $

* $ C(w_1,w_2) $ : bigram count
* $ C(w_1) $ : unigram count
* $ 0 \leq\alpha \leq1 $ :  smoothing hyper-parameter
* |V|: vocabulary size

In [ ]:
# We should fine-tune alpha on a held-out dataset
alpha = 0.01

# Calculate vocab size
# vocab_size = len(set(tokens))
vocab_size = len(vocab)

In [ ]:
vocab_size

271

In [ ]:
# Bigram prob + laplace smoothing
bigram_prob = (bigram_counter[('the', 'evening')] + alpha) / (unigram_counter[('the')] + alpha*vocab_size)
print("bigram_prob: {0:.3f} ".format(bigram_prob))

bigram_log_prob = math.log2(bigram_prob)
print("bigram_log_prob: {0:.3f}".format(bigram_log_prob) )

bigram_prob: 0.004 
bigram_log_prob: -8.082


#  Calculate tri-gram probability

### $ P(w_3|w_1,w_2) = \frac{C(w_1,w_2,w_3) + \alpha}{C(w_1,w_2) + \alpha \cdot|V|} $
Where:

$ C(w_1, w_2, w_3) $: Trigram count, the number of times the trigram (w1, w2, w3) appears in the corpus.
$ C(w_1, w_2) $: Bigram count, the number of times the bigram (w1, w2) appears in the corpus.
$ 0 \leq \alpha \leq 1 $: Smoothing hyper-parameter.
$ |V| $: Vocabulary size, the total number of unique words in the corpus.

In [ ]:
# Triagram prob + laplace smoothing
triagram_prob = (trigram_counter[('the', 'evening','is')] + alpha) / (bigram_counter[('the','evening')] + alpha*vocab_size)
print("triagram_prob: {0:.3f} ".format(bigram_prob))

triagram_log_prob = math.log2(triagram_prob)
print("triagram_log_prob: {0:.3f}".format(triagram_log_prob) )

triagram_prob: 0.004 
triagram_log_prob: -8.082


## Test subset

We have to tokenize our test subset in tokens.

In [ ]:
test_text

"'but i must be on the UNK for UNK UNK and day ! why , i who turned UNK and UNK . UNK , i think you 'd UNK UNK off , ' said the gryphon : and alice was the mock turtle UNK UNK , and began , in a voice UNK UNK it UNK ? i -- ' UNK when they UNK in the UNK . UNK the mouse to tell them something more . UNK her UNK UNK , and the UNK UNK UNK . 'and what are they made of ? ' alice asked in a tone of great UNK . UNK it UNK away without UNK , but at last it UNK its then they UNK UNK UNK , and their UNK got UNK UNK . thing before , and he was n't going to UNK at his time of UNK . turned the UNK , but the rabbit was no UNK to be UNK : she found UNK to UNK at once to eat some of the other bit . her UNK was UNK as all that . ' they had not UNK UNK before they saw the mock turtle in the UNK , UNK , ' said alice : ' -- UNK 's the duchess ? ' it UNK an UNK UNK , no UNK , and very UNK and UNK alice UNK her UNK , and began : -- UNK of UNK UNK that UNK UNK UNK her . 'and just as i 'd UNK the UNK UNK in t

In [ ]:
test_sentences = sent_tokenize(test_text)

for sent in test_sentences:
    print(sent)
    print("_________________")

'but i must be on the UNK for UNK UNK and day !
_________________
why , i who turned UNK and UNK .
_________________
UNK , i think you 'd UNK UNK off , ' said the gryphon : and alice was the mock turtle UNK UNK , and began , in a voice UNK UNK it UNK ?
_________________
i -- ' UNK when they UNK in the UNK .
_________________
UNK the mouse to tell them something more .
_________________
UNK her UNK UNK , and the UNK UNK UNK .
_________________
'and what are they made of ? '
_________________
alice asked in a tone of great UNK .
_________________
UNK it UNK away without UNK , but at last it UNK its then they UNK UNK UNK , and their UNK got UNK UNK .
_________________
thing before , and he was n't going to UNK at his time of UNK .
_________________
turned the UNK , but the rabbit was no UNK to be UNK : she found UNK to UNK at once to eat some of the other bit .
_________________
her UNK was UNK as all that . '
_________________
they had not UNK UNK before they saw the mock turtle in the U

In [ ]:
from nltk import word_tokenize
from nltk import WhitespaceTokenizer
from nltk.tokenize import TweetTokenizer

tweet_wt = TweetTokenizer()

test_sentences_tokenized = []

for sent in test_sentences:
    sent_tok = tweet_wt.tokenize(sent)
    test_sentences_tokenized.append(sent_tok)
    print(sent_tok)
    print("_________________")


["'", 'but', 'i', 'must', 'be', 'on', 'the', 'UNK', 'for', 'UNK', 'UNK', 'and', 'day', '!']
_________________
['why', ',', 'i', 'who', 'turned', 'UNK', 'and', 'UNK', '.']
_________________
['UNK', ',', 'i', 'think', 'you', "'", 'd', 'UNK', 'UNK', 'off', ',', "'", 'said', 'the', 'gryphon', ':', 'and', 'alice', 'was', 'the', 'mock', 'turtle', 'UNK', 'UNK', ',', 'and', 'began', ',', 'in', 'a', 'voice', 'UNK', 'UNK', 'it', 'UNK', '?']
_________________
['i', '-', '-', "'", 'UNK', 'when', 'they', 'UNK', 'in', 'the', 'UNK', '.']
_________________
['UNK', 'the', 'mouse', 'to', 'tell', 'them', 'something', 'more', '.']
_________________
['UNK', 'her', 'UNK', 'UNK', ',', 'and', 'the', 'UNK', 'UNK', 'UNK', '.']
_________________
["'", 'and', 'what', 'are', 'they', 'made', 'of', '?', "'"]
_________________
['alice', 'asked', 'in', 'a', 'tone', 'of', 'great', 'UNK', '.']
_________________
['UNK', 'it', 'UNK', 'away', 'without', 'UNK', ',', 'but', 'at', 'last', 'it', 'UNK', 'its', 'then', 'they', '

In [ ]:
len(test_sentences_tokenized)

252

## Bi-gram LM Cross entropy & perplexity

* $ CrossEntropy = -\frac{1}{N}\sum^{bigrams}{log_2(P(w_2|w_1))} $

* N: Number of bigrams
* $ Perplexity = 2^{H(p)} $

In [ ]:
from itertools import pairwise

sum_prob = 0
bigram_cnt = 0

for sent in sentences_tokenized:
    sent = ['<start>'] + sent + ['<end>']

    # Iterate over the bigrmas of the sentence
    for first_token, second_token in pairwise(sent):
        bigram_prob = (bigram_counter[(first_token, second_token)] + alpha) / (unigram_counter[(first_token,)] + alpha*vocab_size)
        sum_prob += math.log2(bigram_prob)
        bigram_cnt += 1

HC = -sum_prob / bigram_cnt
bi_perpl = math.pow(2,HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(bi_perpl))

Cross Entropy: 3.672
perplexity: 12.742


In [ ]:
# list(pairwise(sent))

## Tri-gram LM Cross entropy & perplexity

* $ CrossEntropy = -\frac{1}{N}\sum_{trigrams} log_2(P(w_3|w_1, w_2)) $
* N: Number of bigrams
* $ Perplexity = 2^{H(p)} $

In [ ]:
from more_itertools import windowed

sum_prob = 0
trigram_cnt = 0

for sent in sentences_tokenized:
    sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']

    for first_token, second_token, third_token in windowed(sent, n=3):
        trigram_prob = (trigram_counter[(first_token, second_token, third_token)] + alpha) / (bigram_counter[(first_token, second_token)] + alpha*vocab_size)
        sum_prob += math.log2(trigram_prob)
        trigram_cnt+=1

HC = -sum_prob / trigram_cnt
tri_perpl = math.pow(2,HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(tri_perpl))

Cross Entropy: 2.432
perplexity: 5.397


## Interpolated bi-gram and tri-gram LM
### $ P(w_3|w_1,w_2) = \lambda \cdot P(w_3|w_1,w_2) +(1-\lambda) \cdot P(w_3|w_2)  $
* $ 0 \leq \lambda \leq 1 $
* $ \lambda $ should be fine-tuned on a held-out dataset
---
$ CrossEntropy = -\frac{1}{N}\sum^{trigrmas}{log_2(P(w_3|w_1,w_2))} = -\frac{1}{N}\sum^{trigrmas}{log_2(\lambda \cdot P(w_3|w_1,w_2) +(1-\lambda) \cdot P(w_3|w_2))}$
* N: Number of trigrams <br>

$ Perplexity = 2^{H(p)} $

In [ ]:
lamda = 0.0001
sum_prob = 0
ngram_cnt = 0

for sent in sentences_tokenized:
    sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']

    for first_token, second_token, third_token in windowed(sent, n=3):

        trigram_prob = (trigram_counter[(first_token, second_token, third_token)] + alpha) / (bigram_counter[(first_token, second_token)] + alpha*vocab_size)
        bigram_prob = (bigram_counter[(second_token, third_token)] + alpha) / (unigram_counter[(second_token,)] + alpha*vocab_size)

        sum_prob += math.log2(lamda * trigram_prob + (1-lamda) * bigram_prob)
        ngram_cnt += 1

        # print('P({}| {}, {})'.format(sent[idx],sent[idx-2], sent[idx-1]))
        # print('P({}| {})'.format(sent[idx], sent[idx-1]))

HC = -sum_prob / ngram_cnt
int_perpl = math.pow(2, HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(int_perpl))

Cross Entropy: 4.149
perplexity: 17.744


In [ ]:
from tabulate import tabulate

data = [
    ["Bi-gram Perplexity", bi_perpl],
    ["Tri-gram Perplexity", tri_perpl],
    ["Interpolation Perplexity", int_perpl],
]

# Create the table
table = tabulate(data, headers=["Variable" , "Value"], tablefmt="grid")

# Print the table
print(table)

+--------------------------+----------+
| Variable                 |    Value |
+==========================+==========+
| Bi-gram Perplexity       | 12.7423  |
+--------------------------+----------+
| Tri-gram Perplexity      |  5.39691 |
+--------------------------+----------+
| Interpolation Perplexity | 17.7439  |
+--------------------------+----------+


As we can observe, the tri-gram model has the lowest perplexity. Lower perplexity values indicate that the model is better at predicting the data

# Hyperparameters Tuning

Tuning is necessary to determine the optimal alpha and lambda values by selecting the settings that provide the best model performance. We are going to use our validation test to do so.

In [ ]:
valid_text

"all of them UNK UNK . he was an old UNK , he was . ' UNK soup ! who UNK for UNK , the UNK UNK ; there were UNK of them , and the little UNK came UNK : 'but i do n't UNK . UNK did they UNK the UNK never before UNK a rabbit with UNK a UNK , or a UNK for some UNK the UNK court was in UNK , getting the dormouse without even UNK to put his UNK on . UNK , but i UNK at a UNK UNK , ' said the dormouse : UNK in that the gryphon added UNK , let 's UNK some of your UNK . ' same when i got up this UNK ? i UNK think i can UNK UNK a UNK . UNK 's UNK at first was UNK . but the UNK of his about the UNK ! ' began UNK UNK about like that ! ' UNK , they were getting so UNK off ) . 'oh , my poor little feet , i UNK on . ' oh ! wo n't she be UNK if i 've UNK her UNK ! ' alice felt so things are `` much of a UNK '' -- did you ever see such a thing as a UNK , i think ? ' tried to get her head down to them , and was UNK to find that her her UNK , and had come to the UNK UNK , that UNK you go and UNK a little

In [ ]:
tweet_wt = TweetTokenizer()

valid_sentences = sent_tokenize(valid_text)
for sent in valid_sentences:
    print(sent)
    print("_________________")

all of them UNK UNK .
_________________
he was an old UNK , he was . '
_________________
UNK soup !
_________________
who UNK for UNK , the UNK UNK ; there were UNK of them , and the little UNK came UNK : 'but i do n't UNK .
_________________
UNK did they UNK the UNK never before UNK a rabbit with UNK a UNK , or a UNK for some UNK the UNK court was in UNK , getting the dormouse without even UNK to put his UNK on .
_________________
UNK , but i UNK at a UNK UNK , ' said the dormouse : UNK in that the gryphon added UNK , let 's UNK some of your UNK . '
_________________
same when i got up this UNK ?
_________________
i UNK think i can UNK UNK a UNK .
_________________
UNK 's UNK at first was UNK .
_________________
but the UNK of his about the UNK ! '
_________________
began UNK UNK about like that ! '
_________________
UNK , they were getting so UNK off ) .
_________________
'oh , my poor little feet , i UNK on . '
_________________
oh !
_________________
wo n't she be UNK if i 've UNK 

In [ ]:
valid_text_tokenized = []
# valid_sentences = sent_tokenize(valid_text)

for sent in valid_sentences:
    sent_tok = tweet_wt.tokenize(sent)
    valid_text_tokenized.append(sent_tok)
    print(sent_tok)
    print("_________________")

['all', 'of', 'them', 'UNK', 'UNK', '.']
_________________
['he', 'was', 'an', 'old', 'UNK', ',', 'he', 'was', '.', "'"]
_________________
['UNK', 'soup', '!']
_________________
['who', 'UNK', 'for', 'UNK', ',', 'the', 'UNK', 'UNK', ';', 'there', 'were', 'UNK', 'of', 'them', ',', 'and', 'the', 'little', 'UNK', 'came', 'UNK', ':', "'", 'but', 'i', 'do', "n't", 'UNK', '.']
_________________
['UNK', 'did', 'they', 'UNK', 'the', 'UNK', 'never', 'before', 'UNK', 'a', 'rabbit', 'with', 'UNK', 'a', 'UNK', ',', 'or', 'a', 'UNK', 'for', 'some', 'UNK', 'the', 'UNK', 'court', 'was', 'in', 'UNK', ',', 'getting', 'the', 'dormouse', 'without', 'even', 'UNK', 'to', 'put', 'his', 'UNK', 'on', '.']
_________________
['UNK', ',', 'but', 'i', 'UNK', 'at', 'a', 'UNK', 'UNK', ',', "'", 'said', 'the', 'dormouse', ':', 'UNK', 'in', 'that', 'the', 'gryphon', 'added', 'UNK', ',', 'let', "'", 's', 'UNK', 'some', 'of', 'your', 'UNK', '.', "'"]
_________________
['same', 'when', 'i', 'got', 'up', 'this', 'UNK', '

In [ ]:
valid_text_tokenized

[['all', 'of', 'them', 'UNK', 'UNK', '.'],
 ['he', 'was', 'an', 'old', 'UNK', ',', 'he', 'was', '.', "'"],
 ['UNK', 'soup', '!'],
 ['who',
  'UNK',
  'for',
  'UNK',
  ',',
  'the',
  'UNK',
  'UNK',
  ';',
  'there',
  'were',
  'UNK',
  'of',
  'them',
  ',',
  'and',
  'the',
  'little',
  'UNK',
  'came',
  'UNK',
  ':',
  "'",
  'but',
  'i',
  'do',
  "n't",
  'UNK',
  '.'],
 ['UNK',
  'did',
  'they',
  'UNK',
  'the',
  'UNK',
  'never',
  'before',
  'UNK',
  'a',
  'rabbit',
  'with',
  'UNK',
  'a',
  'UNK',
  ',',
  'or',
  'a',
  'UNK',
  'for',
  'some',
  'UNK',
  'the',
  'UNK',
  'court',
  'was',
  'in',
  'UNK',
  ',',
  'getting',
  'the',
  'dormouse',
  'without',
  'even',
  'UNK',
  'to',
  'put',
  'his',
  'UNK',
  'on',
  '.'],
 ['UNK',
  ',',
  'but',
  'i',
  'UNK',
  'at',
  'a',
  'UNK',
  'UNK',
  ',',
  "'",
  'said',
  'the',
  'dormouse',
  ':',
  'UNK',
  'in',
  'that',
  'the',
  'gryphon',
  'added',
  'UNK',
  ',',
  'let',
  "'",
  's',
  'UNK',

## Alpha Value Tuning

We train the model on a training subset with the current alpha and calculates cross-entropy on a validation subset for each alpha value. The goal is to identify the alpha value that results in the lowest cross-entropy, indicating the best model performance. The selected "best_alpha" is the one that minimizes the cross-entropy, making it a suitable hyperparameter for the language model.

In [ ]:
# Define the range of alpha values to test
alpha_values = [0.1, 0.01, 0.001, 0.0001]

best_alpha = None
best_cross_entropy = float('inf')

exclude_start = True  # Set this flag to True to exclude "<start>" tokens

for alpha in alpha_values:

    sum_prob = 0
    bigram_cnt = 0

    for sent in valid_text_tokenized:
        sent = ['<start>'] + sent + ['<end>']

        for idx in range(1, len(sent)):
            bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] + alpha) / (unigram_counter[(sent[idx-1],)] + alpha * vocab_size)
            sum_prob += math.log(bigram_prob)
            bigram_cnt += 1

    HC = -sum_prob / bigram_cnt

    if HC < best_cross_entropy:
        best_cross_entropy = HC
        best_alpha = alpha

# Print the best hyperparameter value
print("Best Alpha:", best_alpha)


Best Alpha: 0.01


## Lambda Value Tuning

We follow the same procedure for the lambda value

In [ ]:
# Define the range of lambda values to test
lambda_values = [0.1, 0.01, 0.001, 0.0001]

best_lambda = None
best_cross_entropy = float('inf')

exclude_start = True  # Set this flag to True to exclude "<start1>" and "<start2>" tokens

for lambd in lambda_values:
    # Train the model with the current lambda value on the training subset
    # Calculate interpolated bigram probabilities and cross-entropy on the validation subset
    # Adjust the following code according to your specific language modeling approach

    sum_prob = 0
    bigram_cnt = 0

    for sent in valid_text_tokenized:
        sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']

        for idx in range(1, len(sent)):
            # if sent[idx] in ['<start1>', '<start2>'] and exclude_start:
            #     continue
            # Calculate interpolated bigram probability
            trigram_prob = (trigram_counter[(sent[idx-2],sent[idx-1], sent[idx])] +alpha) / (bigram_counter[(sent[idx-2],sent[idx-1])] + alpha*vocab_size)
            bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] +alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocab_size)

            sum_prob += math.log(bigram_prob)  # Use math.log instead of math.log2
            bigram_cnt += 1

    HC = -sum_prob / bigram_cnt

    if HC < best_cross_entropy:
        best_cross_entropy = HC
        best_lambda = lambd

# Print the best lambda value
print("Best Lambda:", best_lambda)


Best Lambda: 0.1


# After tuning

With the tuned hyperparameters, we can expect improved model performance and better predictive accuracy.

## Bi-gram LM Cross entropy & perplexity

In [ ]:
alpha = best_alpha
sum_prob = 0
bigram_cnt = 0


for sent in test_sentences_tokenized:
    sent = ['<start>'] + sent + ['<end']

    # Iterate over the bigrams of the sentence
    for idx in range(1, len(sent)):
        bigram_prob = (bigram_counter[(sent[idx - 1], sent[idx])] + alpha) / (unigram_counter[(sent[idx - 1],)] + alpha * vocab_size)
        sum_prob += math.log(bigram_prob)
        bigram_cnt += 1

HC = -sum_prob / bigram_cnt
bi_perpl1 = math.pow(2, HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("Perplexity: {0:.3f}".format(bi_perpl1))


Cross Entropy: 3.472
Perplexity: 11.093


In [ ]:
bigram_counter

Counter({('<start>', 'left'): 1,
         ('left', 'UNK'): 3,
         ('UNK', '!'): 160,
         ('!', "'"): 214,
         ("'", '<end>'): 490,
         ('<start>', 'UNK'): 264,
         ('UNK', 'with'): 57,
         ('with', 'the'): 28,
         ('the', 'queen'): 53,
         ('queen', ','): 13,
         (',', 'but'): 37,
         ('but', 'she'): 11,
         ('she', 'UNK'): 77,
         ('UNK', 'that'): 62,
         ('that', 'it'): 20,
         ('it', 'might'): 6,
         ('might', 'UNK'): 7,
         ('UNK', 'any'): 4,
         ('any', 'minute'): 1,
         ('minute', ','): 4,
         (',', 'UNK'): 277,
         ('UNK', 'right'): 4,
         ('right', 'into'): 2,
         ('into', 'it'): 4,
         ('it', '.'): 21,
         ('.', '<end>'): 469,
         ('<start>', "'"): 105,
         ("'", 'that'): 23,
         ('that', "'"): 26,
         ("'", 's'): 138,
         ('s', 'very'): 6,
         ('very', 'curious'): 4,
         ('curious', '!'): 1,
         ('<start>', 'she'): 40,

## Tri-gram LM Cross entropy & perplexity

In [ ]:
alpha = best_alpha
sum_prob = 0
trigram_cnt = 0

for sent in test_sentences_tokenized:
    sent = ['<start1>'] + ['<start2>'] + sent + ['<end']

    for idx in range(2, len(sent) - 1):
        trigram_prob = (trigram_counter[(sent[idx - 2], sent[idx - 1], sent[idx])] + alpha) / (bigram_counter[(sent[idx - 2], sent[idx - 1])] + alpha * vocab_size)
        sum_prob += math.log(trigram_prob)
        trigram_cnt += 1

HC = -sum_prob / trigram_cnt
tri_perpl1 = math.pow(2, HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("Perplexity: {0:.3f}".format(tri_perpl1))


Cross Entropy: 3.425
Perplexity: 10.742


## Interpolated bi-gram and tri-gram LM

In [ ]:
alpha = best_alpha
lamda = best_lambda
sum_prob = 0
ngram_cnt = 0
exclude_start = True

for sent in test_sentences_tokenized:
    sent = ['<start1>'] + ['<start2>'] + sent + ['<end>']

    for idx in range(2, len(sent) - 1):
        trigram_prob = (trigram_counter[(sent[idx-2], sent[idx-1], sent[idx])] + alpha) / (bigram_counter[(sent[idx-2], sent[idx-1])] + alpha * vocab_size)
        bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] + alpha) / (unigram_counter[(sent[idx-1],)] + alpha * vocab_size)

        sum_prob += math.log(lamda * trigram_prob + (1 - lamda) * bigram_prob)
        ngram_cnt += 1

HC = -sum_prob / ngram_cnt
int_perpl1 = math.pow(2, HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(int_perpl1))


Cross Entropy: 3.053
perplexity: 8.302


In [ ]:
from tabulate import tabulate

data = [
    ["Bi-gram Perplexity",bi_perpl ,bi_perpl1],
    ["Tri-gram Perplexity",tri_perpl, tri_perpl1],
    ["Interpolation Perplexity", int_perpl,int_perpl1],
]

# Create the table
table = tabulate(data, headers=["Before Tuning","After Tuning"], tablefmt="grid")

# Print the table
print(table)

+--------------------------+-----------------+----------------+
|                          |   Before Tuning |   After Tuning |
+==========================+=================+================+
| Bi-gram Perplexity       |        12.7423  |       11.0928  |
+--------------------------+-----------------+----------------+
| Tri-gram Perplexity      |         5.39691 |       10.7418  |
+--------------------------+-----------------+----------------+
| Interpolation Perplexity |        17.7439  |        8.30205 |
+--------------------------+-----------------+----------------+


Based on the table provided, it is evident that our most effective model is the untuned trigram model.

# Generate sentence completions
We are going to produce sentence completions by choosing the most likely word at each step until the generated sentence concludes with the end token.

## Bigram Model

In [ ]:
#the second word in the first bigram, is the first one in the second
bigram_counter

Counter({('<start>', 'left'): 1,
         ('left', 'UNK'): 3,
         ('UNK', '!'): 160,
         ('!', "'"): 214,
         ("'", '<end>'): 490,
         ('<start>', 'UNK'): 264,
         ('UNK', 'with'): 57,
         ('with', 'the'): 28,
         ('the', 'queen'): 53,
         ('queen', ','): 13,
         (',', 'but'): 37,
         ('but', 'she'): 11,
         ('she', 'UNK'): 77,
         ('UNK', 'that'): 62,
         ('that', 'it'): 20,
         ('it', 'might'): 6,
         ('might', 'UNK'): 7,
         ('UNK', 'any'): 4,
         ('any', 'minute'): 1,
         ('minute', ','): 4,
         (',', 'UNK'): 277,
         ('UNK', 'right'): 4,
         ('right', 'into'): 2,
         ('into', 'it'): 4,
         ('it', '.'): 21,
         ('.', '<end>'): 469,
         ('<start>', "'"): 105,
         ("'", 'that'): 23,
         ('that', "'"): 26,
         ("'", 's'): 138,
         ('s', 'very'): 6,
         ('very', 'curious'): 4,
         ('curious', '!'): 1,
         ('<start>', 'she'): 40,

In [ ]:
#the second word in the first trigrram, is the first one in the second
trigram_counter

Counter({('<start1>', '<start2>', 'left'): 1,
         ('<start2>', 'left', 'UNK'): 1,
         ('left', 'UNK', '!'): 1,
         ('UNK', '!', "'"): 111,
         ('!', "'", '<end>'): 205,
         ('<start1>', '<start2>', 'UNK'): 264,
         ('<start2>', 'UNK', 'with'): 5,
         ('UNK', 'with', 'the'): 11,
         ('with', 'the', 'queen'): 1,
         ('the', 'queen', ','): 13,
         ('queen', ',', 'but'): 1,
         (',', 'but', 'she'): 5,
         ('but', 'she', 'UNK'): 2,
         ('she', 'UNK', 'that'): 4,
         ('UNK', 'that', 'it'): 6,
         ('that', 'it', 'might'): 3,
         ('it', 'might', 'UNK'): 4,
         ('might', 'UNK', 'any'): 1,
         ('UNK', 'any', 'minute'): 1,
         ('any', 'minute', ','): 1,
         ('minute', ',', 'UNK'): 3,
         (',', 'UNK', 'right'): 1,
         ('UNK', 'right', 'into'): 2,
         ('right', 'into', 'it'): 1,
         ('into', 'it', '.'): 1,
         ('it', '.', '<end>'): 14,
         ('<start1>', '<start2>', "'"): 

As evident, our models contain unidentified tokens (UNKs), which pose a challenge when predicting certain words. Furthermore, it's noteworthy that UNK is the most prevalent token. Consequently, we have opted to eliminate all UNKs from our models.

In [ ]:
# Initialize a new bigram counter without bigrams containing "UNK"
filtered_bigram_counter = {}

for bigram, count in bigram_counter.items():
    # Check if "UNK" is not in any of the words in the bigram
    if 'UNK' not in bigram:
        filtered_bigram_counter[bigram] = count

# Now, filtered_bigram_counter contains only the bigrams without "UNK"
filtered_bigram_counter

{('<start>', 'left'): 1,
 ('!', "'"): 214,
 ("'", '<end>'): 490,
 ('with', 'the'): 28,
 ('the', 'queen'): 53,
 ('queen', ','): 13,
 (',', 'but'): 37,
 ('but', 'she'): 11,
 ('that', 'it'): 20,
 ('it', 'might'): 6,
 ('any', 'minute'): 1,
 ('minute', ','): 4,
 ('right', 'into'): 2,
 ('into', 'it'): 4,
 ('it', '.'): 21,
 ('.', '<end>'): 469,
 ('<start>', "'"): 105,
 ("'", 'that'): 23,
 ('that', "'"): 26,
 ("'", 's'): 138,
 ('s', 'very'): 6,
 ('very', 'curious'): 4,
 ('curious', '!'): 1,
 ('<start>', 'she'): 40,
 ('she', 'thought'): 12,
 ('thought', '.'): 2,
 ("'", 'but'): 30,
 ('but', "'"): 3,
 ("'", 'i'): 90,
 ('i', 'do'): 26,
 ('do', "n't"): 39,
 ("n't", 'like'): 6,
 ('like', 'the'): 9,
 ('the', 'look'): 3,
 ('look', 'of'): 2,
 ('of', 'it'): 9,
 ('it', 'at'): 3,
 ('at', 'all'): 16,
 ('all', ','): 6,
 (',', "'"): 367,
 ("'", 'said'): 156,
 ('said', 'the'): 148,
 ('the', 'king'): 39,
 ('king', ':'): 2,
 (',', 'it'): 23,
 ('it', 'was'): 57,
 ('was', "n't"): 7,
 ("n't", 'very'): 2,
 ('of', '

In [ ]:
def bi_generate_sentence(initial_phrase):
    initial_words = initial_phrase.split()  # Tokenize the initial phrase
    last_word = initial_words[-1]


    if (initial_words[0], initial_words[1]) not in filtered_bigram_counter:
      # Find the bigram with the highest probability in the filtered_bigram_counter
      random_bigram = max(list(filtered_bigram_counter.keys()))
      initial_words += random_bigram

      # Check if '<end>' is not already in initial_words, and add it
      if '<end>' not in initial_words:
          initial_words.append('<end>')
      else:
          return ' '.join(initial_words)


    else:
         # Generate the sentence until "<end>" is encountered
        while True:
            # Find possible bigrams starting with the last word
            possible_bigrams = [bigram for bigram in filtered_bigram_counter if bigram[0] == last_word]

            if not possible_bigrams:
                break  # If no valid bigrams, break the loop

            # Select a random bigram based on their counts
            next_bigram = max(possible_bigrams, [filtered_bigram_counter[bigram] for bigram in possible_bigrams])[0]

            # Append the second word of the selected bigram to initial_words
            initial_words.append(next_bigram[1])

            # Update last_word for the next iteration
            last_word = next_bigram[1]

            # If the second word is '<end>', stop generating
            if next_bigram[1] == '<end>':
                break

  # Join the generated words into a complete sentence
    generated_sentence = ' '.join(initial_words)

    return generated_sentence


In [ ]:
# Example usage
initial_phrase = 'this is'

generated_sentence = bi_generate_sentence(initial_phrase)
print("Generated sentence:", generated_sentence)


Generated sentence: this is your two <end>


In [ ]:
# Example usage
initial_phrase = 'this unknownword'


generated_sentence = bi_generate_sentence(initial_phrase)
print("Generated sentence:", generated_sentence)


Generated sentence: this unknownword your two <end>


## Trigram Model

In [ ]:
# Initialize a new bigram counter without bigrams containing "UNK"
filtered_trigram_counter = {}

for trigram, count in trigram_counter.items():
    # Check if "UNK" is not in any of the words in the trigram
    if 'UNK' not in trigram:
        filtered_trigram_counter[trigram] = count

# Now, filtered_trigram_counter contains only the trigrams without "UNK"
filtered_trigram_counter

{('<start1>', '<start2>', 'left'): 1,
 ('!', "'", '<end>'): 205,
 ('with', 'the', 'queen'): 1,
 ('the', 'queen', ','): 13,
 ('queen', ',', 'but'): 1,
 (',', 'but', 'she'): 5,
 ('that', 'it', 'might'): 3,
 ('any', 'minute', ','): 1,
 ('right', 'into', 'it'): 1,
 ('into', 'it', '.'): 1,
 ('it', '.', '<end>'): 14,
 ('<start1>', '<start2>', "'"): 105,
 ('<start2>', "'", 'that'): 5,
 ("'", 'that', "'"): 12,
 ('that', "'", 's'): 25,
 ("'", 's', 'very'): 6,
 ('s', 'very', 'curious'): 2,
 ('very', 'curious', '!'): 1,
 ('curious', '!', "'"): 1,
 ('<start1>', '<start2>', 'she'): 40,
 ('<start2>', 'she', 'thought'): 3,
 ('she', 'thought', '.'): 2,
 ('thought', '.', '<end>'): 1,
 ('<start2>', "'", 'but'): 4,
 ("'", 'but', "'"): 1,
 ('but', "'", 'i'): 2,
 ("'", 'i', 'do'): 12,
 ('i', 'do', "n't"): 18,
 ('do', "n't", 'like'): 3,
 ("n't", 'like', 'the'): 1,
 ('like', 'the', 'look'): 3,
 ('the', 'look', 'of'): 2,
 ('look', 'of', 'it'): 1,
 ('of', 'it', 'at'): 2,
 ('it', 'at', 'all'): 3,
 ('at', 'all',

In [ ]:
def tri_generate_sentence(initial_phrase):
    initial_words = initial_phrase.split()  # Tokenize the initial phrase

    if (initial_words[0], initial_words[1], initial_words[2]) not in filtered_trigram_counter:
        # Fallback to bigram if the trigram doesn't exist
        return bi_generate_sentence(' '.join(initial_words[:2]))

    else:
        # Generate the sentence until "<end>" is encountered
        while True:
            # Get the last two words in initial_words
            last_two_words = initial_words[-2:]

            # Find possible trigrams starting with the last two words
            possible_trigrams = [trigram for trigram in filtered_trigram_counter if trigram[0] == last_two_words[0] and trigram[1] == last_two_words[1]]

            if not possible_trigrams:
                break  # If no valid trigrams, break the loop

            # Select a random trigram based on their counts
            next_trigram = random.choices(possible_trigrams, [filtered_trigram_counter[trigram] for trigram in possible_trigrams])[0]

            # Append the third word of the selected trigram to initial_words
            initial_words.append(next_trigram[2])

            # If the third word is '<end>', stop generating
            if next_trigram[2] == '<end>':
                break

        # Join the generated words into a complete sentence
        generated_sentence = ' '.join(initial_words)

        return generated_sentence


In [ ]:
# Example usage
initial_phrase = 'she did not'
generated_sentence = tri_generate_sentence(initial_phrase)
print("Generated sentence:", generated_sentence)

Generated sentence: she did not get


In [ ]:
# Example usage
initial_phrase = 'this is unknownword'
generated_sentence = tri_generate_sentence(initial_phrase)
print("Generated sentence:", generated_sentence)

Generated sentence: this is your two <end>



A trigram model is generally preferred over a bigram model for text generation due to its ability to consider a more extended context. While a bigram model looks at the preceding word when predicting the next word, a trigram model takes into account the two preceding words, which offers a richer and more detailed understanding of language.

## Test sentences

In [ ]:
test_text

"'but i must be on the UNK for UNK UNK and day ! why , i who turned UNK and UNK . UNK , i think you 'd UNK UNK off , ' said the gryphon : and alice was the mock turtle UNK UNK , and began , in a voice UNK UNK it UNK ? i -- ' UNK when they UNK in the UNK . UNK the mouse to tell them something more . UNK her UNK UNK , and the UNK UNK UNK . 'and what are they made of ? ' alice asked in a tone of great UNK . UNK it UNK away without UNK , but at last it UNK its then they UNK UNK UNK , and their UNK got UNK UNK . thing before , and he was n't going to UNK at his time of UNK . turned the UNK , but the rabbit was no UNK to be UNK : she found UNK to UNK at once to eat some of the other bit . her UNK was UNK as all that . ' they had not UNK UNK before they saw the mock turtle in the UNK , UNK , ' said alice : ' -- UNK 's the duchess ? ' it UNK an UNK UNK , no UNK , and very UNK and UNK alice UNK her UNK , and began : -- UNK of UNK UNK that UNK UNK UNK her . 'and just as i 'd UNK the UNK UNK in t

Our goal is to reconstruct the sentences present in the "test_text." To achieve this, we replace each comma separator with a full stop.

In [ ]:
# Replace spaces with commas
text_with_commas = test_text.replace(" ,", ".")

# Print the result
print(text_with_commas)

'but i must be on the UNK for UNK UNK and day ! why. i who turned UNK and UNK . UNK. i think you 'd UNK UNK off. ' said the gryphon : and alice was the mock turtle UNK UNK. and began. in a voice UNK UNK it UNK ? i -- ' UNK when they UNK in the UNK . UNK the mouse to tell them something more . UNK her UNK UNK. and the UNK UNK UNK . 'and what are they made of ? ' alice asked in a tone of great UNK . UNK it UNK away without UNK. but at last it UNK its then they UNK UNK UNK. and their UNK got UNK UNK . thing before. and he was n't going to UNK at his time of UNK . turned the UNK. but the rabbit was no UNK to be UNK : she found UNK to UNK at once to eat some of the other bit . her UNK was UNK as all that . ' they had not UNK UNK before they saw the mock turtle in the UNK. UNK. ' said alice : ' -- UNK 's the duchess ? ' it UNK an UNK UNK. no UNK. and very UNK and UNK alice UNK her UNK. and began : -- UNK of UNK UNK that UNK UNK UNK her . 'and just as i 'd UNK the UNK UNK in the UNK. ' UNK th

In [ ]:
sentences = text_with_commas.split('.')
random.shuffle(sentences)
test_sentences = sentences[:10]
text_with_commas = '.'.join(test_sentences)
print(text_with_commas)

 UNK UNK were UNK UNK any UNK a UNK of UNK for UNK UNK up like UNK : this to take out of it. said the dormouse ; ' -- well in . the mouse only UNK in UNK . the UNK ) could not make out UNK `` i UNK by his garden . UNK UNK they wo n't UNK the way i UNK UNK. ' said the duchess. and UNK. and thought it would go by the UNK. just at UNK -- at UNK i know UNK. who UNK made a UNK is n't any


In [ ]:
test_sentences[0]

' UNK UNK were UNK UNK any UNK a UNK of UNK for UNK UNK up like UNK : this to take out of it'

We need to demonstrate, through the presentation of some sample generated texts, that the trigram model produces more coherent and fluent text compared to the bigram model.

# Context - Aware Spelling Corrector

This Context - Aware Spelling Corrector will utilize my trigram language model, employ a beam search decoder for better candidate selection, and incorporate the mathematical formulas from [slide 19](https://eclass.aueb.gr/modules/document/file.php/INF210/slides_2023_24/nlp_slides_part01_ngrams_spelling_correction.pdf) to make accurate corrections.

Given a text input, you need to identify words that are potentially misspelled. This can be done using techniques like Levenshtein distance, phonetic similarity, or a dictionary lookup against a known list of correctly spelled words.

In [ ]:
Levenshtein =  nltk.edit_distance

For each identified misspelled word, create a set of candidate corrections by applying common edit operations. These operations include:

*   Insertion: Add a single character at any position in the word.
*   Deletion: Remove a single character from the word.
*   Substitution: Replace a single character with another character.
*   Transposition: Swap adjacent characters in the word.

In [ ]:
def generate_candidates(word, max_distance):
    # Generate candidates based on Levenshtein distance
    candidates = [candidate for candidate in vocab if Levenshtein(word, candidate) <= max_distance]
    return candidates

In [ ]:
def score(candidate, word):
    levenshtein_score = -Levenshtein(candidate, word)
    trigram_score = filtered_trigram_counter.get((word, candidate), 0)
    return levenshtein_score + trigram_score

In [ ]:
def beam_search_decode(initial_sentence, max_distance, beam_width, generate_candidates, score):
    words = initial_sentence.split()  # Split the sentence into words
    corrected_words = []

    for word in words:
        sequences = [([word], 0)]  # Initial sequence with a score of 0

        for _ in range(beam_width):  # Perform two iterations (you can adjust this)
            candidates = []
            for seq, seq_score in sequences:
                last_word = seq[-1]
                for next_word in generate_candidates(last_word, max_distance):
                    new_seq = seq + [next_word]
                    new_score = seq_score + score(next_word, last_word)
                    candidates.append((new_seq, new_score))

            # Select the top candidates based on score
            candidates.sort(key=lambda x: x[1], reverse=True)
            if candidates:
                sequences = candidates[:beam_width]
            else:
                # Handle the case when no candidates are available
                sequences = [([last_word], seq_score)]  # Revert to the last word as the only option

        best_correction = sequences[0][0][-1]  # Get the best correction for the word
        corrected_words.append(best_correction)

    corrected_sentence = " ".join(corrected_words)
    return corrected_sentence


In [ ]:
# Misspelled sentence
initial_sentence = "ws cred mch"
max_distance = 5  # Maximum Levenshtein distance allowed
beam_width = 2  # Number of candidates to consider

# Find the best correction for the sentence
best_correction = beam_search_decode(initial_sentence, max_distance, beam_width, generate_candidates, score)

print("Corrected Sentence:", best_correction)

Corrected Sentence: 's cried much


In [ ]:
# Misspelled sentence
initial_sentence = "wts cred mch"
max_distance = 5  # Maximum Levenshtein distance allowed
beam_width = 2  # Number of candidates to consider

# Find the best correction for the sentence
best_correction = beam_search_decode(initial_sentence, max_distance, beam_width, generate_candidates, score)

print("Corrected Sentence:", best_correction)

Corrected Sentence: was cried much


In [ ]:
# Misspelled sentence
initial_sentence = "mie tio"
max_distance = 5  # Maximum Levenshtein distance allowed
beam_width = 2  # Number of candidates to consider

# Find the best correction for the sentence
best_correction = beam_search_decode(initial_sentence, max_distance, beam_width, generate_candidates, score)

print("Corrected Sentence:", best_correction)

Corrected Sentence: me to


#  Evaluate the Context-Aware Spelling Corrector

The misspelling_probability is used to control how frequently misspellings are introduced into the sentences when creating the artificial test dataset. In the provided code, misspelling_probability is set to 0.05, which means there is a 5% chance that any given character in a sentence will be replaced with a misspelled character.

In [ ]:
# Define the misspelling probability (e.g., 0.05 for 5%)
misspelling_probability = 0.05

In [ ]:
replacement_rules = {
    'a': 'e',
    'b': 'v',
    'c': 's',
    'd': 't',
    'e': 'i',
    'f': 'g',
    'g': 'j',
    'h': 'n',
    'i': 'o',
    'j': 'k',
    'k': 'l',
    'l': 'm',
    'm': 'n',
    'n': 'm',
    'o': 'p',
    'p': 'o',
    'q': 'w',
    'r': 't',
    's': 'd',
    't': 'y',
    'u': 'y',
    'v': 'b',
    'w': 'q',
    'x': 'z',
    'y': 'u',
    'z': 'x',
    ',': '.',
    '.': ',',
    ' ': ' ',
}


In [ ]:
def introduce_misspellings(sentence):
    misspelled_sentence = ''
    for char in sentence:
        if char != ' ' and random.random() <= misspelling_probability:
            if char == 'U' and sentence.startswith("UNK"):
                misspelled_sentence += 'y'
            else:
                misspelled_sentence += replacement_rules.get(char, char)
        else:
            misspelled_sentence += char
    return misspelled_sentence

In [ ]:
test_sentences = [introduce_misspellings(sentence) for sentence in test_set]

In [ ]:
misspelled_dataset = [introduce_misspellings(sentence) for sentence in test_sentences]

In [ ]:
def correct_spelling_with_beam_search(misspelled_sentence, max_distance, beam_width, generate_candidates, score):
    max_distance = 5  # Maximum Levenshtein distance allowed
    beam_width = 2  # Number of candidates to consider
    corrected_sentence = beam_search_decode(misspelled_sentence, max_distance, beam_width, generate_candidates, score)
    return corrected_sentence

In [ ]:
# Shuffle the misspelled dataset
random.shuffle(misspelled_dataset)

# Keep the first 10 sentences
subset_of_sentences = misspelled_dataset[:20]
subset_of_sentences = [sentence.strip() for sentence in subset_of_sentences]
# Filter out empty sentences (if needed)
subset_of_sentences = [sentence for sentence in subset_of_sentences if sentence]

# Now, 'subset_of_sentences' contains a shuffled subset of 10 sentences from the misspelled dataset.
subset_of_sentences

['woumd not, could not, would not, coult npt, would not join thi dance.',
 'frijhtened mouse soladhed nis way yhroygh tne neighbouting poom--dhe',
 "'give your evidence,' said the king; 'end dom't be nervous, or i'll neve",
 "it twelve? p--'",
 'desperate that she was ready to ask helo of any one; sp, when the ravbit',
 'doqn in a large arm-chair at one end of the tabli.',
 'to take out of it, and burning with curiodity, she tan across the gield',
 "tine,' dhe said, 'than waste ot in asking riddmes that have no answers.'",
 'alice, whp felt riady to sink into yhi eatth. at last the gryphon saod',
 'vlow underneath her chin: it had strucl her foot!',
 'when the pie wes ell finished, the owl, as a boon,',
 'pne elbow againsy yhe door, amd the oyner arm curled round het hied,',
 "about e whiting begore.'",
 'they had not gone far before they saw thi nock tyryle on the distance,']

In [ ]:
corrected_dataset = [correct_spelling_with_beam_search(sentence, max_distance, beam_width, generate_candidates, score) for sentence in subset_of_sentences]
# Remove leading and trailing spaces from each sentence
corrected_dataset = [sentence.strip() for sentence in corrected_dataset]

# Filter out empty sentences (if needed)
corrected_dataset = [sentence for sentence in corrected_dataset if sentence]

corrected_dataset

["would not could not would not could n't would not down the once",
 'frijhtened mouse added is way thought the neighbouting poom--dhe',
 "'ve your evidence,' said the king 'and do be curious or 'll never",
 'it table --',
 'desperate that she was head to as her of any one , when the rabbit',
 'down in a large arm-chair at one and of the table',
 'to take out of it and being with curious she an curious the did',
 "time the said 'that was it in king time that have no answers.'",
 'alice who felt any to king into the hatter at last the gryphon said',
 'how under her thing it had such her look',
 'when the the was all find the only as a soon',
 'one look again the door and the one are turned round her did',
 'about ! white before',
 'they had not tone for before they saw the mock turtle on the distance,']

In [ ]:
from tabulate import tabulate

# Create a list of lists for the table
table_data = []
for misspelled, corrected in zip(subset_of_sentences, corrected_dataset):
    table_data.append([misspelled, corrected])

# Display the table
table = tabulate(table_data, headers=["Misspelled Sentence", "Corrected Sentence"], tablefmt="grid")
print(table)


+--------------------------------------------------------------------------+------------------------------------------------------------------------+
| Misspelled Sentence                                                      | Corrected Sentence                                                     |
+==========================================================================+========================================================================+
| woumd not, could not, would not, coult npt, would not join thi dance.    | would not could not would not could n't would not down the once        |
+--------------------------------------------------------------------------+------------------------------------------------------------------------+
| frijhtened mouse soladhed nis way yhroygh tne neighbouting poom--dhe     | frijhtened mouse added is way thought the neighbouting poom--dhe       |
+--------------------------------------------------------------------------+------------------------

# Metrics

##  Word Error Rate (WER)

WER is a metric used to evaluate the performance of systems that generate or correct sequences of words, such as automatic speech recognition systems or spelling correction systems. It measures the number of word-level errors made by a system when comparing its output to the reference or ground truth text.

### $ WER = \frac{S + I + D}{N} $

* $ S $ : the number of substitutions
* $ i $ : the number of insertions
* $ D $ : the number of deletions
* N: the total number of words in the reference

In [ ]:
def calculate_wer(reference, corrected):
    # Calculate Word Error Rate (WER)
    reference_words = reference.split()
    corrected_words = corrected.split()

    S = Levenshtein(reference_words, corrected_words)
    I = max(0, len(corrected_words) - len(reference_words))
    D = max(0, len(reference_words) - len(corrected_words))

    N = max(len(reference_words), len(corrected_words))

    wer = (S + I + D) / N

    return wer


In [ ]:
total_wer = 0
num_sentences = len(subset_of_sentences)

for reference, corrected in zip(subset_of_sentences, corrected_dataset):
    wer = calculate_wer(reference, corrected)
    total_wer += wer

average_wer = total_wer / num_sentences

print("Average WER:", average_wer)

Average WER: 0.5437764616336044


## Character Error Rate (CER)
CER is similar to WER but operates at the character level. It measures the accuracy of character-level transcriptions or corrections. CER quantifies the number of character-level errors made by a system when comparing its output to the reference text. Errors include substitutions, insertions, and deletions of individual characters.

### $ WER = \frac{S + I + D}{N} $

* $ S $ : the number of character substitutions
* $ i $ : the number of character insertions
* $ D $ : the number of character deletions
* N: the total number of characterσ in the reference

In [ ]:
def calculate_cer(reference, corrected):
    S = Levenshtein(reference, corrected)
    I = max(0, len(corrected) - len(reference))
    D = max(0, len(reference) - len(corrected))

    N = max(len(reference), len(corrected))

    cer = (S + I + D) / N

    return cer


In [ ]:
# Assuming you have corrected_dataset and subset_of_sentences

total_cer = 0
num_sentences = len(subset_of_sentences)

for reference, corrected in zip(subset_of_sentences, corrected_dataset):
    # Calculate CER
    cer = calculate_cer(reference, corrected)
    total_cer += cer

average_cer = total_cer / num_sentences

print("Average CER:", average_cer)


Average CER: 0.31843751821435423


In both WER and CER, a lower error rate indicates better accuracy

In [ ]:
from tabulate import tabulate

data = [
    ["Average WER", average_wer],
    ["Average CER", average_cer]
]

# Create the table
table = tabulate(data, headers=["Metrics" , "Value"], tablefmt="grid")

# Print the table
print(table)

+-------------+----------+
| Metrics     |    Value |
+=============+==========+
| Average WER | 0.543776 |
+-------------+----------+
| Average CER | 0.318438 |
+-------------+----------+
